In [5]:
import numpy as np
import pandas as pd
from sam_fun import *

In [16]:
#add your api key
api_key = ""
from googleapiclient.discovery import build
youtube = build("youtube","v3",developerKey=api_key)

In [7]:
#top creators from india's trending video page
#link to other country codes
#https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2 
in_creator_ids= trending_creators_by_country(youtube,"IN")

In [8]:
#dataframe of stats from creators, made with the list of ids
in_stats_df = channels_stats(youtube,in_creator_ids)

In [9]:
in_stats_df

,title,country,viewCount,subscriberCount,videoCount,channel_id
0,ashish chanchlani vines,IN,4287295613,29400000,149,UC7eHZXheF8nVOfwB2PEslMw
1,Saregama Music,IN,14021634891,32800000,6866,UC_A7K2dXFsTMAciGmnNxy-Q
2,khordha toka,IN,114231061,836000,67,UCmuSPlchjLraieuEs33mcVA
3,Ajay Devgn Ffilms,IN,365240260,2090000,157,UChkXkESv2ViSatGUdIN9gBw
4,Round2World,IN,153458464,1450000,130,UChCy9K0jbV525iS5W9poBVA
...,...,...,...,...,...,...
146,Racha Gang,IN,70163684,463000,188,UC3st0eH7gCWwg2wqaYa1GDg
147,Nav Haryanvi,IN,3184656074,9130000,755,UCbYXVxwDEWc9sIt6yZhweqQ
148,Harsha Sai - For You Tamil,IN,485116704,4430000,79,UCmpIo1MVMCSyNaf1rRQS8RA
149,ALBUM CREATIONS,IN,1244361259,2670000,389,UC232UhFsjFYeMJdP5nd80RQ


In [10]:
# gets top 5% of channels based off subscribers
in_95 = top_channels(in_stats_df,"subscriberCount",95)

In [13]:
# gets id of every video on khordha toka channel
video_ids = get_videoID_list(youtube,"UCmuSPlchjLraieuEs33mcVA" ) # or in_stats_df[2] 
                 
                 

In [14]:
# dataframe containing stats of all videos of a channel
get_video_details(youtube,video_ids)

,title,published,description,tags,view_count,dislike_count,comment_count
0,Ama Gaan huli khela || Khordha toka || Funny a...,2023-03-06T11:27:24Z,Download probo 👇\nhttps://probo-in.onelink.me/...,"[ama gaan huli khela, khordha toka comedy, ama...",794463,private,2188
1,Chala jiba picnic || FULL VDO || BHAI THE MOV...,2023-01-06T14:23:38Z,Brand promotion by :- @the_lucky_goyal \nKhord...,"[chala jiba picnic, khordhatoka, funny anuguli...",3026120,private,3282
2,chala jiba PICNIC || TEASER || Khordha toka |...,2022-12-26T10:36:34Z,,"[khordha toka, Anguilla, mr dhenkanaliaa, khor...",196798,private,296
3,Ama ghara katha PART3 || Khordha toka || Funny...,2022-11-05T12:03:54Z,Download probo 👇\nhttps://probo-in.onelink.me/...,"[ama ghara katha, khordha toka comedy, funny a...",4088504,private,2384
4,Puja re lagila 144 || Khordha toka || Funny an...,2022-09-21T10:21:25Z,Click for FREE registration on Olymp Trade: ht...,"[ganesh puja impossible, khordha toka comedy, ...",1706208,private,3080
...,...,...,...,...,...,...,...
63,Ame Samaste ek//. Odia motivational video //CR...,2018-02-12T10:06:30Z,,"[ame samaste ek, Ame samaste ek, AME SAMASTE E...",36661,private,190
64,YAARA TERI YAARI..😢😢// CRaZyyy BOYZZ,2018-01-17T08:00:44Z,,"[yaara teri yaari, YAARA TERI YAARI, Yaara Ter...",21721,private,163
65,ODIA Advertisement VS REALITY || By CRazzY B...,2018-01-02T10:59:47Z,,"[Odia advertisement vs reality, odia ADVERTISE...",110591,private,166
66,BAPA || a. Heart touching short film||,2017-12-19T08:38:06Z,,[],25978,private,205
